# Analysis of GFP dilution 

In [26]:
import numpy as npj
import pandas as pd
import bokeh.io
import bokeh.plotting
import glob
import pymc3 as pm
import theano.tensor as tt
import scipy.optimize
import sys
sys.path.insert(0, '../../../')
import mwc.viz
import mwc.process
colors = mwc.viz.set_plotting_style()
bokeh.io.output_notebook()

Loading BokehJS ...

In [17]:
# Load all of the clist files for the growth experiment.
data_dir = '../../../data/images/20180206_sfGFP_mscL_dilution/'
growth_clist = glob.glob('{0}/*growth*/xy*/clist.mat'.format(data_dir))
growth_df = mwc.process.parse_clists(growth_clist)

In [18]:
# Load the autofluorescence and mlg910
samps = ['autofluorescence', 'mlg910']
dfs = []
for i, s in enumerate(samps):
    clists = glob.glob('{0}/*{1}*/xy*/clist.mat'.format(data_dir, s))
    _df = mwc.process.parse_clists(clists, added_props={'strain':s})
    dfs.append(_df)
snap_df = pd.concat(dfs, ignore_index=True)

In [19]:
# Compute the mean autofluorescence
mean_auto_gfp = snap_df[snap_df['strain']=='autofluorescence']['fluor1_mean_death'].mean()

In [20]:
mean_auto_gfp

308.9851078426133

Now, isolate the cells in which the fluorescence was actually measured.

In [21]:
# Look at only the measured cells
measured = growth_df[growth_df['fluor1_mean_death'] > 0]


# Set up the calibration dataframe.
cal_df = pd.DataFrame([], columns=['I_1', 'I_2', 'summed', 'fluct'])
# Group the cells by position and mother ID for calculation.
grouped = growth_df.groupby(['position', 'mother_id'])
for g, d in grouped:
    if len(d) == 2:
        daughters = d['area_death'].values * (d['fluor1_mean_death'].values - mean_auto_gfp)
        if (daughters > 0).all():
            I_1, I_2 = daughters
            summed = np.sum(daughters)
            fluct = np.diff(daughters)[0]**2
            cell_dict = dict(I_1=I_1, I_2=I_2, summed=summed, fluct=fluct)
            cal_df = cal_df.append(cell_dict, ignore_index=True)

In [ ]:
def deterministic_log_post(alpha, i1, i2, neg=True):
    if neg:
        prefactor = 

In [54]:
def deterministic_log_post(alpha, i1, i2, neg=True):
    if neg:
        prefactor = -1
    else:
        prefactor = 1
    if alpha < 1E-9:
        return prefactor * -np.inf
    n1 = i1 / alpha
    n2 = i2 / alpha
    ntot = n1 + n2
    k = len(n1)
    binom = scipy.special.gammaln(ntot+1).sum() - scipy.special.gammaln(n1+1).sum() -\
                scipy.special.gammaln(n2+1).sum()
    lp = -k * np.log(alpha) - ntot.sum() * np.log(2) + binom
    return prefactor * lp
    

In [57]:
# Minimize teh log posterior given the data. 
popt = scipy.optimize.minimize_scalar(deterministic_log_post, args=(cal_df.I_1, cal_df.I_2, True))

In [58]:
popt

     fun: 4682.5942389145785
    nfev: 14
     nit: 13
 success: True
       x: 621.32650708632798

In [61]:
# Generate the plot.
I_tot_range = np.logspace(2, 5, 500)
theo = popt.x * I_tot_range
p = mwc.viz.boilerplate(plot_width=800, x_axis_label='summed intensity [a.u.]',
                       y_axis_label='squared fluctuation', x_axis_type='log',
                       y_axis_type='log')

p.circle('summed', 'fluct', source=cal_df, size=3, color='slategray')
p.line(I_tot_range, theo, color='tomato')
bokeh.io.show(p)

In [68]:
mlg910 = snap_df[snap_df['strain']=='mlg910']
mean_mlg910 = np.mean((mlg910['area_death'] - mean_auto_gfp) * mlg910['fluor1_mean_death'])

In [69]:
mean_mlg910 / (5 * popt.x)

23.821720243081977